In [1]:
import torch
from torchvision.transforms import v2
from torch.utils.data import DataLoader

import project.utils
%reload project
from project.data.luna_dataset import Luna16Dataset
from project.models.vnet import VNet

In [ ]:
model = VNet(num_classes=1)


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

In [16]:
from torch import nn
from torchvision.transforms import v2
from torchvision import tv_tensors
from project.config import PROJECT_ROOT

class PadDepthTransform(nn.Module):
    """Pad the depth dimension of the input tensor to be divisible by 16."""
    def forward(self, img, mask) -> tuple[torch.Tensor, torch.Tensor]:
        # Check if the number of depth dimensions is odd
        if img.shape[1] % 16 != 0:
            # Create a zero-filled padding with the same height and width
            padding = torch.zeros(1, 16 - img.shape[1] % 16, *img.shape[2:], device=img.device, dtype=img.dtype)
            # Concatenate the padding to the tensor
            img = torch.cat([img, padding], dim=1)
            mask = torch.cat([mask, padding], dim=1)
        return tv_tensors.Image(img), tv_tensors.Mask(mask)

transforms = v2.Compose([
    v2.Resize((80, 80)),
    PadDepthTransform(),
    v2.ToDtype({tv_tensors.Image: torch.float32, tv_tensors.Mask: torch.int64, "others": None}, scale=True),
])

luna16 = Luna16Dataset(root=PROJECT_ROOT / "data/luna16", transforms=transforms, train=True)
luna16_base = Luna16Dataset(root=PROJECT_ROOT / "data/luna16", transforms=None, train=True)

In [17]:
transforms(*luna16_base[0])

(Image([[[[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          ...,
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923]],
 
         [[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          ...,
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923]],
 
         [[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923,

In [18]:
len(luna16.masks)

712

In [19]:
luna16[0][0].shape, luna16[0][1].shape

(torch.Size([1, 128, 80, 80]), torch.Size([1, 128, 80, 80]))

In [20]:
luna_train_loader = DataLoader(luna16, batch_size=1, shuffle=False)  # Batch size has to be 1 because each image has different depth

In [21]:
for i, (data, target) in enumerate(luna_train_loader):
    # print(data.shape, data.dtype)
    # print(target.shape, target.dtype)
    print(i)
    if target.dtype != torch.int64:
        print(target.dtype)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 

In [15]:
from project.models.vnet import VNet
from project.models.unet3d import UNet3D
import segmentation_models_3D as sm

# model = VNet(num_classes=1)
# model = UNet3D(n_channels=1, n_classes=1)
model = sm.FPN(
    'densenet121',
    classes=1,
    activation='sigmoid'
)

ModuleNotFoundError: No module named 'tensorflow'

In [10]:
# Number of parameters
sum(p.numel() for p in model.parameters() if p.requires_grad)

2353601

In [13]:
# Train loop

import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.train().to(device)
for epoch in range(10):
    for i, (data, target) in enumerate(luna_train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        print(output.shape, target.shape)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")
    scheduler.step()


torch.Size([1, 1]) torch.Size([1, 1, 128, 80, 80])


ValueError: Target size (torch.Size([1, 1, 128, 80, 80])) must be the same as input size (torch.Size([1, 1]))

In [ ]:
data.shape

In [ ]:
data.shape